In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

f_modele_path = "Models/"
f_modele_lite_path = 'Models/_Lite/'

In [5]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(f_modele_path + "WaveNet2DConv0.1-16-Batch-56-epoch-30")
tflite_model = converter.convert()

# Save the model.
with open(f_modele_lite_path + 'WaveNet0.5.2.tflite', 'wb') as f:
  f.write(tflite_model)

In [14]:
df = pd.read_feather('PrepedData/LPG-Profile-4-plus-exp.feather')

In [16]:
df.energy.max(), df.energy.min()

(8.106128643591507, 0.12007945580992319)

In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model(f_modele_path + "WaveNet2DConv0.1-16-Batch-56-epoch-30")

# Convert the model to the TensorFlow Lite format with quantization
with open("PrepedData/Test/LPG4XTrain0.4.npy", "rb") as f:
    XTrain = np.load(f)

def representative_dataset():
    for X in XTrain:
      data = X.reshape(1,1,40,4)
      yield [data.astype(np.float32)]
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
# #                                        tf.lite.OpsSet.SELECT_TF_OPS]
# # converter.experimental_new_converter = True
# # Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(f_modele_lite_path + 'WaveNet0.5.2quantized.tflite', "wb").write(model_tflite)

33928